In [1]:

%load_ext autoreload
%autoreload 2


In [2]:
import os
os.chdir('../')

import sys 
sys.path.append('../')
import time
import pandas as pd 
import numpy as np
import torch
import openai
from tqdm import tqdm
from dotenv import load_dotenv
from trainer.transformer_utilts import *
from helper.dataloader import * 
import pickle
from peft import get_peft_model
from model.MF import sentenceT5Classification
import argparse
from tqdm import tqdm
from peft import LoraConfig, TaskType
import logging
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoTokenizer


logging.basicConfig(filename=f'./logs/{time.time()}.log',level=logging.INFO)
logging.info(f'Experiment started at {time.time()}')


def parse_args(notebook = False):  # Parse command line arguments
    parser = argparse.ArgumentParser(description="LLM4RecSys")
    parser.add_argument("--data_name", default='ml-1m', type=str)
    parser.add_argument("--direction", default='down', type=str)
    parser.add_argument("--bs", default=32, type=int)
    parser.add_argument("--target_index", default=49, type=int)
    parser.add_argument("--num_samples", default=250, type=int)
    parser.add_argument("--max_l", default=260, type=int)
    parser.add_argument("--debug_prompts", default=False, type=bool)
    parser.add_argument("--metadata_path",default='./data_preprocessed/books/train.csv',type=str)
    args = parser.parse_args() if not notebook else parser.parse_args(args=[])
    args.scratch = '/home/mila/e/emiliano.penaloza/scratch'
    return args
args = parse_args(True)
args.data_name = 'books'


load_dotenv()
max_l = args.max_l
key = os.getenv("OPEN-AI-SECRET")
openai.api_key = key
tokenizer = AutoTokenizer.from_pretrained("t5-large")
prompts,rec_dataloader,num_movies,val_dataloader,test_dataloader,_,_= load_data(args,tokenizer,0,1)

# Load model and mapping dicts
path = f'{args.scratch}/saved_model/{args.data_name}/t5_classification_embedding_module_t5_classification_l2_lambda_0.0_lora_r_32_scheduler_cosine_warmup_0.001_0.2.csv.pt'
model = sentenceT5Classification.from_pretrained('t5-large', num_labels=num_movies)

lora_config = LoraConfig(task_type=TaskType.SEQ_CLS, r=32, lora_alpha=16, lora_dropout=0,
                            target_modules=["q", "v"],
                            modules_to_save=['classification_head'])
model = get_peft_model(model, lora_config)
model.load_state_dict(torch.load(path ,map_location=torch.device('cpu')))
model.to(0)


/home/mila/e/emiliano.penaloza/llm4rec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mila/e/emiliano.penaloza/llm4rec/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model

data_path='./data_preprocessed/books/'
Max Prompt Length 260
Number of Users is num_users=3534
Number of Movies is num_movies=14871


Some weights of sentenceT5Classification were not initialized from the model checkpoint at t5-large and are newly initialized: ['classification_head.dense.weight', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): sentenceT5Classification(
      (transformer): T5EncoderModel(
        (shared): Embedding(32128, 1024)
        (encoder): T5Stack(
          (embed_tokens): Embedding(32128, 1024)
          (block): ModuleList(
            (0): T5Block(
              (layer): ModuleList(
                (0): T5LayerSelfAttention(
                  (SelfAttention): T5Attention(
                    (q): lora.Linear(
                      (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                      (lora_dropout): ModuleDict(
                        (default): Identity()
                      )
                      (lora_A): ModuleDict(
                        (default): Linear(in_features=1024, out_features=32, bias=False)
                      )
                      (lora_B): ModuleDict(
                        (default): Linear(in_features=32, out_features=1024, bias=False)
                    

In [ ]:
example = '''
Summary: The user has a distinct preference for fiction, particularly works that delve into science fiction, fantasy, and speculative narratives, \
    often with a touch of epic storytelling or significant historical context. \
    These genres resonate due to their imaginative scope and the ability to transport the audience to different realms or alternate realities,\
    offering complex world-building and often featuring iconic characters on journeys or missions with high stakes.\
    The user appreciates plots revolving around heroism, adventure, and warfare, both tactical and grandiose in scale, particularly those that interweave fictional\
    elements with historical nods or futuristic settings. In contrast, the user might not favor genres that are grounded strictly\
    in the current reality or those that focus on the more mundane aspects of life. For example, works strictly categorized under technology & engineering might \
    be less appealing if they lack the narrative drive or the philosophical underpinnings found in the user's preferred genres.\
    Autobiographies and strictly narrative-driven real-world historical events, without the infusion of dramatization or thematic depth, could potentially fall flat for the user,\
        despite their appreciation for history when it intertwines with fiction or grand storytelling.'''
s = '''
Summary: The user has a distinct preference for fiction, particularly works that delve into science fiction, and speculative narratives, \
    often with a touch of epic storytelling or significant historical context. \
    These genres resonate due to their imaginative scope and the ability to transport the audience to different realms or alternate realities,\
    offering complex world-building and often featuring iconic characters on journeys or missions with high stakes.\
    The user appreciates plots revolving around heroism, adventure, and warfare, both tactical and grandiose in scale, particularly those that interweave fictional\
    elements with  futuristic settings. In contrast, the user might not favor genres that are grounded strictly\
    in the current reality or those that focus on the more mundane aspects of life. For example, works strictly categorized under technology & engineering might \
    be less appealing if they lack the narrative drive or the philosophical underpinnings found in the user's preferred genres.\
    Autobiographies and strictly narrative-driven real-world historical events, without the infusion of dramatization or thematic depth, could potentially fall flat for the user,\
        despite their appreciation for history when it intertwines with fiction or grand storytelling.
'''
example_item = 'Star Wars: I, Jedi'

In [3]:
example = '''
Summary: The user has a distinct preference for fiction, particularly works that delve into science fiction, fantasy, and speculative narratives, \
    often with a touch of epic storytelling or significant historical context. \
    These genres resonate due to their imaginative scope and the ability to transport the audience to different realms or alternate realities,\
    offering complex world-building and often featuring iconic characters on journeys or missions with high stakes.\
    The user appreciates plots revolving around heroism, adventure, and warfare, both tactical and grandiose in scale, particularly those that interweave fictional\
    elements with historical nods or futuristic settings. In contrast, the user might not favor genres that are grounded strictly\
    in the current reality or those that focus on the more mundane aspects of life. For example, works strictly categorized under technology & engineering might \
    be less appealing if they lack the narrative drive or the philosophical underpinnings found in the user's preferred genres.\
    Autobiographies and strictly narrative-driven real-world historical events, without the infusion of dramatization or thematic depth, could potentially fall flat for the user,\
        despite their appreciation for history when it intertwines with fiction or grand storytelling.'''
s = '''
Summary: The user has a distinct preference for fiction, particularly works that delve into fantasy, and speculative narratives, \
    often with a touch of epic storytelling or significant historical context. \
    These genres resonate due to their imaginative scope and the ability to transport the audience to different realms or alternate realities.\
    The user appreciates plots revolving around heroism, adventure, both tactical and grandiose in scale, particularly those that interweave fictional\
    elements with historical nods. In contrast, the user might not favor genres that are grounded strictly\
    in the current reality or those that focus on the more mundane aspects of life. For example, works strictly categorized under technology & engineering might \
    be less appealing if they lack the narrative drive or the philosophical underpinnings found in the user's preferred genres.\
    Autobiographies and strictly narrative-driven real-world historical events, without the infusion of dramatization or thematic depth, could potentially fall flat for the user,\
        despite their appreciation for history when it intertwines with fiction or grand storytelling.
'''
example_item = 'Star Wars: I, Jedi'

In [ ]:
example

"\nSummary: The user has a distinct preference for fiction, particularly works that delve into science fiction, fantasy, and speculative narratives,     often with a touch of epic storytelling or significant historical context.     These genres resonate due to their imaginative scope and the ability to transport the audience to different realms or alternate realities,    offering complex world-building and often featuring iconic characters on journeys or missions with high stakes.    The user appreciates plots revolving around heroism, adventure, and warfare, both tactical and grandiose in scale, particularly those that interweave fictional    elements with historical nods or futuristic settings. In contrast, the user might not favor genres that are grounded strictly    in the current reality or those that focus on the more mundane aspects of life. For example, works strictly categorized under technology & engineering might     be less appealing if they lack the narrative drive or the 

In [ ]:
original = pd.read_csv(f'./results/{args.data_name}/gpt4_results_{taget_rank}_{direction}.csv')

In [ ]:
original.to_csv(f'./results/{args.data_name}/gpt4_results_{taget_rank}_{direction}_old.csv')

In [4]:

with open(f'./data_preprocessed/{args.data_name}/show2id.pkl','rb') as f:
    movie_id_map = pickle.load(f)
    movie_id_map = {v:k for k,v in movie_id_map.items()}
    


    

item_id_to_title = map_id_to_title( args.metadata_path,data = 'books')
item_id_to_genre = map_id_to_genre(args.metadata_path,data = 'books')


#easy tokenization function 
def tokenize_prompt(tokenizer,prompt,max_l):
    encodings = tokenizer([prompt],padding=True, truncation=True,max_length=max_l,
                          return_tensors='pt')
    return encodings

# gets diffs in recommendations between two summaries
def get_difs(s1,s2,device = 0,target_rank = 10 ):
    rankings = []
    genres = []
    s_1_encodings = tokenize_prompt(tokenizer,s1,max_l)
    s_2_encodings = tokenize_prompt(tokenizer,s2,max_l)
    for k in s_1_encodings:
        s_1_encodings[k] = s_1_encodings[k].to(device)
    for k in s_2_encodings:
        s_2_encodings[k] = s_2_encodings[k].to(device)
    out_s1 = model(**s_1_encodings)
    out_s2 = model(**s_2_encodings)
    l = torch.topk(torch.softmax(out_s1.logits,dim = 1),2000)
    indices_1 = l.indices[0]
    l2 = torch.topk(torch.softmax(out_s2.logits,dim = 1),2000)
    indices_2 = l2.indices[0]
    target_index_s1 = indices_1[target_rank].item()
    pos = torch.where(indices_2 == target_index_s1)[0].item()
    delta = target_rank - pos
    for i in range(len(indices_2)):
            index_2 = indices_2[i].item()
            movie_id2 = movie_id_map[index_2] if args.data_name != 'books' else index_2
            genres.append(item_id_to_genre[movie_id2])
            rankings.append(item_id_to_title[movie_id2])
    return delta,pos,rankings,genres

#get recommendations for a single summary
def get_recs(s1,device = 0,ranking = 10): 
    s = ''
    rankings = []
    genres = []
    s1_encodings = tokenize_prompt(tokenizer,s1,max_l)
    #move encodings to device
    for k in s1_encodings:
        s1_encodings[k] = s1_encodings[k].to(device)
    out_s1 = model(**s1_encodings)
    l = torch.topk(torch.softmax(out_s1.logits,dim = 1),1000)
    indices_1 = l.indices[0]
    for i in range(len(indices_1)):
            index_1 = indices_1[i].item()
            movie_id1 = movie_id_map[index_1] if args.data_name != 'books' else index_1
            movie_title = item_id_to_title[movie_id1]
            rankings.append(item_id_to_title[movie_id1])
            genres.append(item_id_to_genre[movie_id1])
            if i == ranking:
                target_movie = movie_title

    return target_movie,s,rankings ,genres

np.random.seed(2024)
sampled_keys = np.random.choice(list(prompts.keys()),args.num_samples , replace=False)
#shuffle the list 


gpt_outputs_d = defaultdict(list)
original_rankings = defaultdict(list)
changed_rankings = defaultdict(list)
original_genres = defaultdict(list)
changed_genres = defaultdict(list)
original_prompts = {}
user_target_movie ={}
deltas = {}
running_deltas = []
bleu_scores = {}
running_bleu_scores = []
direction = args.direction
model.eval()
taget_rank = args.target_index

# Prompt GPT to change summaries 
for k in (pbar:=tqdm(sampled_keys)):
    target_movie, ranking,rankings,o_genres  = get_recs(prompts[k],ranking = taget_rank)
    

    rankings =  {"\n".join([f'{i+1,item}' for i,item in enumerate(rankings[:args.target_index+args.target_index])])}
    prompt = \
                f"""
                You will be fed user profiles which summarize user's preferences. Please make edits to them that such that you push
                the input book as far {direction} as possible. Please maintain the summaries in the following format: 
                You will be fed the summaries, the target movie, and the rankings. Please only output the full summaries keep them as close to the input summary as possible.
                Keep the summaries in the following format: \n
                Summary: [Specific details about genres the user enjoys]. [Specific details of plot points the user seems to enjoy]. 
                [Specific details about genres the user does not enjoy]. [Specific details of plot points the user does not enjoy but other users may].\n 
                Example: 
                Here we slightly augment the user's preference to make {example_item} more appealing to the user by removing words not associated with it.\n
                Original summary:\n {example}\n
                Augmented summary: \n {s}\n
                """
    user_prompt =\
                f"""
                The current rankings are:
                {rankings} \n \n
                Taking into account the rankings make changes to the genres and plot points the user enjoys and does not enjoy.\n
                Do not change full sentences but simply replace words or remove them without replacement\n 
                Make the changes to move the target item {direction} and try to keep the rest of the rankings the same\n
                Do not just bold or emphasize words\n 
                Remember do not include any information about the target item in the summary.\n
                Please only output the full summary make edit/remove at most 10-15 words. 
                Target movie to move in ranking {direction}: {target_movie}\n
                {prompts[k]}\n 
                """
    while True:
        try:
            msg = [
                {
                    "role": "system",
                    "content": prompt
                },
                {
                    "role": "user",
                    "content": user_prompt
                }
            ]

            gpt_output = openai.ChatCompletion.create(
                model='gpt-4-1106-preview',
                messages=msg,
                temperature=1,
                max_tokens=300,
                # seed=2024
            )['choices'][0]['message']['content']

            # If the API call is successful and does not throw an error, break the loop
            break
        except Exception as e:
            # If an error occurs, print the error and continue with the next iteration of the loop
            print(f"An error occurred: {e}")
            
    gpt_outputs_d[k].append(gpt_output)
    original_prompts[k] = prompts[k]
    delta, new_rank,new_rankings,generes = get_difs(s1 = prompts[k],s2 = gpt_output,target_rank = taget_rank)
    user_target_movie[k] = target_movie
    original_rankings[k].append(rankings)
    changed_rankings[k].append(new_rankings)
    deltas[k] = delta
    changed_genres[k].append(generes)   
    original_genres[k].append(o_genres)
    running_deltas.append(delta)
    reference = [word for word in prompts[k].split()]
    candidate = [word for word in gpt_output.split()]
    bleu_score = sentence_bleu([reference], candidate)
    bleu_scores[k] = bleu_score
    running_bleu_scores.append(bleu_score)
    deltas_list = list(deltas.values())
    #get the proportion of deltas below zero 
    prop_below_zero = len([i for i in deltas_list if i < 0])/len(deltas_list)
    print(f'Proportion of deltas below zero: {prop_below_zero}',flush = True)
    #get the ones above zer o 
    prop_above_zero = len([i for i in deltas_list if i > 0])/len(deltas_list)
    print(f'Proportion of deltas above zero: {prop_above_zero}',flush = True)


    pbar.set_description(f'Target item: {target_movie}, Original Ranking: Delta: {delta} , Running mean Delta: {np.mean(running_deltas):.3f},Running median Delta: {np.median(running_deltas)}, Mean BLEU Score: {np.mean(running_bleu_scores):.3f}')
    logging.info(f'Target item: {target_movie}, Original Ranking: Delta: {delta} , Running Delta: {np.mean(running_deltas):.3f},Running Delta: {np.median(running_deltas)}, Mean BLEU Score: {np.mean(running_bleu_scores):.3f}')
    
    

#make a dataframe for the dicts 
blue_scores_df = pd.DataFrame.from_dict(bleu_scores,orient = 'index')
gpt_outputs_df = pd.DataFrame.from_dict(gpt_outputs_d,orient = 'index')
original_rankings_df = pd.DataFrame.from_dict(original_rankings,orient = 'index')
changed_rankings_df = pd.DataFrame.from_dict(changed_rankings,orient = 'index')
deltas_df = pd.DataFrame.from_dict(deltas,orient = 'index')
original_prompts_df = pd.DataFrame.from_dict(original_prompts,orient = 'index')

 
#save dataframe 
all_df = pd.concat([original_prompts_df,gpt_outputs_df,original_rankings_df,changed_rankings_df,deltas_df,blue_scores_df],axis = 1)
all_df.columns = ['Original Summary','changed Summary','Original Ranking','Changed Ranking','Delta','blue_scores']

all_df.to_csv(f'./results/{args.data_name}/gpt4_results_{taget_rank}_{direction}.csv')
logging.info(f'Saved results to ./results/{args.data_name}/gpt4_results_{taget_rank}_{direction}.csv')



  0%|          | 0/250 [00:00<?, ?it/s]

Proportion of deltas below zero: 0.0
Proportion of deltas above zero: 1.0


Target item: Star Wars: I, Jedi, Original Ranking: Delta: 15 , Running mean Delta: 15.000,Running median Delta: 15.0, Mean BLEU Score: 0.711:   0%|          | 1/250 [00:10<42:41, 10.29s/it]

Proportion of deltas below zero: 0.5
Proportion of deltas above zero: 0.5


Target item: Ringworld, Original Ranking: Delta: -278 , Running mean Delta: -131.500,Running median Delta: -131.5, Mean BLEU Score: 0.812:   1%|          | 2/250 [00:21<45:47, 11.08s/it]   

...

Target item: Memoirs of a Geisha (Signed), Original Ranking: Delta: -25 , Running mean Delta: -54.972,Running median Delta: -10.0, Mean BLEU Score: 0.813:  99%|█████████▉| 248/250 [31:01<00:26, 13.15s/it] 

Proportion of deltas below zero: 0.6305220883534136
Proportion of deltas above zero: 0.3534136546184739


Target item: Slaughter-House Five, Original Ranking: Delta: 0 , Running mean Delta: -54.751,Running median Delta: -10.0, Mean BLEU Score: 0.814: 100%|█████████▉| 249/250 [31:06<00:10, 10.73s/it]          

Proportion of deltas below zero: 0.628
Proportion of deltas above zero: 0.356


Target item: Moby Dick Or the Whale, Original Ranking: Delta: 3 , Running mean Delta: -54.520,Running median Delta: -10.0, Mean BLEU Score: 0.814: 100%|██████████| 250/250 [31:12<00:00,  7.49s/it]


In [ ]:
original = pd.read_csv(f'./results/{args.data_name}/gpt4_results_{taget_rank}_{direction}_first.csv')
original.Delta.mean()
original.index = original['Unnamed: 0']
original

,Unnamed: 0,Original Summary,changed Summary,Original Ranking,Changed Ranking,Delta,blue_scores
Unnamed: 0,,,,,,,
1675,1675,Summary: The user has a distinct preference fo...,Summary: The user has a distinct preference fo...,"['Star Wars Episode II', 'Star Wars Njo Force ...",['Star Wars Njo Force Heretic Remnant 1 (Star ...,2,0.975552
3094,3094,Summary: The user enjoys a variety of fiction ...,Summary: The user enjoys a variety of fiction ...,"['Starship Troopers', 'Foundation', 'FOUNDATIO...","['I am Legend', 'I Am Legend', 'Snow Crash', '...",19,0.910443
370,370,Summary: The user gravitates towards works roo...,Summary: The user gravitates towards works roo...,"['The Lincoln Lawyer', 'City of Bones', 'Echo ...","['The Lincoln Lawyer', 'City of Bones', 'Echo ...",7,0.775099
2609,2609,Summary: The user shows a clear preference for...,Summary: The user shows a clear preference for...,"['Little Women (Courage giant classics)', 'Lit...","['A Tree Grows in Brooklyn', 'The poisonwood B...",-6,0.832724
1958,1958,Summary: The user revels in expansive fictiona...,Summary: The user revels in expansive fictiona...,"['Dracula (G. K. Hall (Large Print))', 'Treasu...","['Dracula (G. K. Hall (Large Print))', 'Treasu...",0,1.000000
...,...,...,...,...,...,...,...
1146,1146,Summary: The user has a distinctive preference...,Summary: The user has a distinctive preference...,"['Mere Christianity', 'The Great Divorce', 'Th...","['Mere Christianity', 'The Great Divorce', 'Th...",-3,0.774429
198,198,Summary: The user has a pronounced appreciatio...,Summary: The user has a pronounced appreciatio...,"['Nineteen Eighty-four', '1984', 'George Orwel...","['Nineteen Eighty-four', '1984', 'George Orwel...",-23,0.945073
3044,3044,Summary: The user enjoys fiction genres that o...,Summary: The user enjoys fiction genres that o...,"['Five People You Meet in Heaven', ""The Pilot'...","['Five People You Meet in Heaven', ""The Pilot'...",3,0.913799


In [ ]:
# Create a new dataframe to track the changes
changes_df = original.copy()

# Loop through all users
for user in all_df.index:
    # Get the delta values for the user
    delta_all_df = all_df.loc[user, 'Delta']
    delta_original = original.loc[user, 'Delta']
    
    # Check if the delta is positive in all_df but negative in original
    if delta_all_df > 0 and delta_original < 0:
        print(f"{delta_all_df=}")
        # Replace the delta in all_df with the delta in original
        changes_df.loc[user,:] = all_df.loc[user,:] 


# Print the changes dataframe
print(changes_df)


delta_all_df=5
delta_all_df=3
delta_all_df=3
delta_all_df=3
delta_all_df=7
delta_all_df=11
delta_all_df=15
delta_all_df=1
delta_all_df=6
delta_all_df=7
delta_all_df=6
delta_all_df=19
delta_all_df=2
delta_all_df=5
delta_all_df=3
delta_all_df=5
delta_all_df=1
delta_all_df=9
delta_all_df=1
delta_all_df=13
delta_all_df=13
delta_all_df=13
delta_all_df=8
delta_all_df=1
delta_all_df=1
delta_all_df=11
delta_all_df=7
delta_all_df=1
delta_all_df=1
delta_all_df=10
delta_all_df=3
            Unnamed: 0                                   Original Summary  \
Unnamed: 0                                                                  
1675            1675.0  Summary: The user has a distinct preference fo...   
3094            3094.0  Summary: The user enjoys a variety of fiction ...   
370              370.0  Summary: The user gravitates towards works roo...   
2609               NaN  Summary: The user shows a clear preference for...   
1958            1958.0  Summary: The user revels in expansive fic

In [ ]:
all_df

,Original Summary,changed Summary,Original Ranking,Changed Ranking,Delta,blue_scores
1675,Summary: The user has a distinct preference fo...,Summary: The user has a distinct preference fo...,"{(1, 'Star Wars Episode II')\n(2, 'Star Wars N...","[A Feast for Crows (A Song of Ice and Fire, Bo...",16,0.961849
3094,Summary: The user enjoys a variety of fiction ...,Summary: The user enjoys a variety of fiction ...,"{(1, 'Starship Troopers')\n(2, 'Foundation')\n...","[Starship Troopers, Foundation, FOUNDATION, Th...",0,1.000000
370,Summary: The user gravitates towards works roo...,Summary: The user gravitates towards works roo...,"{(1, 'The Lincoln Lawyer')\n(2, 'City of Bones...","[The Lincoln Lawyer, City of Bones, Echo Park ...",0,1.000000
2609,Summary: The user shows a clear preference for...,Summary: The user shows a clear preference for...,"{(1, 'Little Women (Courage giant classics)')\...","[Little women, or, Meg, Jo, Beth, and Amy, Lit...",-245,0.732750
1958,Summary: The user revels in expansive fictiona...,Summary: The user revels in expansive fictiona...,"{(1, 'Dracula (G. K. Hall (Large Print))')\n(2...","[Dracula (G. K. Hall (Large Print)), Treasure ...",0,1.000000
...,...,...,...,...,...,...
1146,Summary: The user has a distinctive preference...,Summary: The user has a distinctive preference...,"{(1, 'Mere Christianity')\n(2, 'The Great Divo...","[Mere Christianity, The Great Divorce, The gre...",-110,0.953606
198,Summary: The user has a pronounced appreciatio...,Summary: The user has a pronounced appreciatio...,"{(1, 'Nineteen Eighty-four')\n(2, '1984')\n(3,...","[Nineteen Eighty-four, 1984, George Orwell 198...",21,0.985643
3044,Summary: The user enjoys fiction genres that o...,Summary: The user enjoys fiction genres that o...,"{(1, 'Five People You Meet in Heaven')\n(2, ""T...","[Five People You Meet in Heaven, The Pilot's W...",11,0.943539
694,Summary: The user enjoys creative nonfiction t...,Summary: The user enjoys creative nonfiction t...,"{(1, 'Great Gatsby (Everyman)')\n(2, 'The grea...","[Great Gatsby (Everyman), The great Gatsby (Le...",0,1.000000


In [ ]:
changes_df

,Unnamed: 0,Original Summary,changed Summary,Original Ranking,Changed Ranking,Delta,blue_scores
Unnamed: 0,,,,,,,
1675,1675.0,Summary: The user has a distinct preference fo...,Summary: The user has a distinct preference fo...,"['Star Wars Episode II', 'Star Wars Njo Force ...",['Star Wars Njo Force Heretic Remnant 1 (Star ...,2,0.975552
3094,3094.0,Summary: The user enjoys a variety of fiction ...,Summary: The user enjoys a variety of fiction ...,"['Starship Troopers', 'Foundation', 'FOUNDATIO...","['I am Legend', 'I Am Legend', 'Snow Crash', '...",19,0.910443
370,370.0,Summary: The user gravitates towards works roo...,Summary: The user gravitates towards works roo...,"['The Lincoln Lawyer', 'City of Bones', 'Echo ...","['The Lincoln Lawyer', 'City of Bones', 'Echo ...",7,0.775099
2609,NaN,Summary: The user shows a clear preference for...,Summary: The user shows a clear preference for...,"{(1, 'Little Women (Courage giant classics)')\...","[Little Women: or Meg, Jo, Beth, and Amy, Litt...",5,0.768159
1958,1958.0,Summary: The user revels in expansive fictiona...,Summary: The user revels in expansive fictiona...,"['Dracula (G. K. Hall (Large Print))', 'Treasu...","['Dracula (G. K. Hall (Large Print))', 'Treasu...",0,1.000000
...,...,...,...,...,...,...,...
1146,1146.0,Summary: The user has a distinctive preference...,Summary: The user has a distinctive preference...,"['Mere Christianity', 'The Great Divorce', 'Th...","['Mere Christianity', 'The Great Divorce', 'Th...",-3,0.774429
198,198.0,Summary: The user has a pronounced appreciatio...,Summary: The user has a pronounced appreciatio...,"['Nineteen Eighty-four', '1984', 'George Orwel...","['Nineteen Eighty-four', '1984', 'George Orwel...",-23,0.945073
3044,3044.0,Summary: The user enjoys fiction genres that o...,Summary: The user enjoys fiction genres that o...,"['Five People You Meet in Heaven', ""The Pilot'...","['Five People You Meet in Heaven', ""The Pilot'...",3,0.913799


In [ ]:
print(f"{f'./results/{args.data_name}/gpt4_results_{taget_rank}_{direction}_chang.csv'=}")


f'./results/{args.data_name}/gpt4_results_{taget_rank}_{direction}_changes.csv'='./results/books/gpt4_results_19_up_changes.csv'


In [ ]:
changes_df.drop(columns = 'Unnamed: 0').to_csv(f'./results/{args.data_name}/gpt4_results_{taget_rank}_{direction}.csv')

In [ ]:
#drop column userId write to the save file without index
changes_df.to_csv(f'./results/{args.data_name}/gpt4_results_{taget_rank}_{direction}.csv',index = False)

In [ ]:
original.Delta.mean()

-9.048

In [ ]:
changes_df.Delta.median()

1.0

In [ ]:
    reference = [word for word in prompts[k].split()]
    candidate = [word for word in gpt_output.split()]
    bleu_score = sentence_bleu([reference], candidate)
    print(f"{bleu_score=}")

bleu_score=0.932442560465694


In [ ]:
original = '''
Summary: The user has a distinct preference for fiction, particularly works that delve into science fiction, fantasy, and speculative narratives, \
    often with a touch of epic storytelling or significant historical context. \
    These genres resonate due to their imaginative scope and the ability to transport the audience to different realms or alternate realities,\
    offering complex world-building and often featuring iconic characters on journeys or missions with high stakes.\
    The user appreciates plots revolving around heroism, adventure, and warfare, both tactical and grandiose in scale, particularly those that interweave fictional\
    elements with historical nods or futuristic settings. In contrast, the user might not favor genres that are grounded strictly\
    in the current reality or those that focus on the more mundane aspects of life. For example, works strictly categorized under technology & engineering might \
    be less appealing if they lack the narrative drive or the philosophical underpinnings found in the user's preferred genres.\
    Autobiographies and strictly narrative-driven real-world historical events, without the infusion of dramatization or thematic depth, could potentially fall flat for the user,\
        despite their appreciation for history when it intertwines with fiction or grand storytelling.'''
s = '''
Summary: The user has a distinct preference for fiction, particularly works that delve into science fiction, and speculative narratives, \
    often with a touch of epic storytelling or significant historical context. \
    These genres resonate due to their imaginative scope and the ability to transport the audience to different realms or alternate realities,\
    offering complex world-building and often featuring iconic characters on journeys or missions with high stakes.\
    The user appreciates plots revolving around heroism, adventure, and warfare, both tactical and grandiose in scale, particularly those that interweave fictional\
    elements with  futuristic settings. In contrast, the user might not favor genres that are grounded strictly\
    in the current reality or those that focus on the more mundane aspects of life. For example, works strictly categorized under technology & engineering might \
    be less appealing if they lack the narrative drive or the philosophical underpinnings found in the user's preferred genres.\
    Autobiographies and strictly narrative-driven real-world historical events, without the infusion of dramatization or thematic depth, could potentially fall flat for the user,\
        despite their appreciation for history when it intertwines with fiction or grand storytelling.
'''

In [ ]:
delta, new_rank,new_rankings,generes = get_difs(s1 = prompts[k],s2 = s,target_rank = taget_rank)
print(f"{delta=}")
new_rank

delta=16


33

In [ ]:
new_rankings[:10]

['Star Wars Episode II',
 'Star Wars Njo Force Heretic Remnant 1 (Star Wars New Jedi Order) (v. 1)',
 'HYPERION',
 'The Crystal Star',
 'A Feast for Crows (A Song of Ice and Fire, Book Four)',
 'A Deepness in the Sky',
 'Starship Troopers',
 'Dune',
 "Star Wars Survivor's Quest",
 'Reunion. Sean Williams and Shane Dix (Star Wars New Jedi Order) (v. 3)']

In [ ]:
new_rankings[:30]

['Star Wars Episode II',
 'Star Wars Njo Force Heretic Remnant 1 (Star Wars New Jedi Order) (v. 1)',
 'HYPERION',
 'The Crystal Star',
 'A Feast for Crows (A Song of Ice and Fire, Book Four)',
 'A Deepness in the Sky',
 'Starship Troopers',
 'Dune',
 "Star Wars Survivor's Quest",
 'Reunion. Sean Williams and Shane Dix (Star Wars New Jedi Order) (v. 3)',
 'A Princess of Mars',
 'FOUNDATION',
 'A princess of Mars',
 'Tales from the Empire: Star Wars',
 'The Hunt for RED OCTOBER',
 'A Canticle for Leibowitz',
 'Foundation',
 'Battle Flag: Starbuck Chronicles, Vol. 3',
 'Hard Contact (Star Wars: Republic Commando, Book 1)',
 'Foundation And Empire',
 'Foundation and Empire',
 'Down to Earth (Colonization, Book 2)',
 'Barrayar (unsigned book club edition)',
 'Slave Ship (Star Wars: The Bounty Hunter Wars, Book 2)',
 'The Martian Way',
 'Battle Flag (Starbuck Chronicles)',
 'House Atreides',
 'Shadow of the Hegemon',
 'Outbound Flight (Star Wars)',
 'The Reality Dysfunction Part I: Emergence

In [ ]:
deltas_list = list(changes_df.Delta)
#get the proportion of deltas below zero 
prop_below_zero = len([i for i in deltas_list if i < 0])/len(deltas_list)
print(f'Proportion of deltas below zero: {prop_below_zero}')
#get the ones above zer o 
prop_above_zero = len([i for i in deltas_list if i > 0])/len(deltas_list)
print(f'Proportion of deltas above zero: {prop_above_zero}')


Proportion of deltas below zero: 0.336
Proportion of deltas above zero: 0.564


In [ ]:
from pprint import pprint
pprint(user_prompt)

In [ ]:
s = set(movie_id_map.keys())
val = set(movie_id_map.values())

In [ ]:
max(list(movie_id_map.keys()))

In [ ]:
import pandas as pd 

In [ ]:
df = pd.read_csv('/home/mila/e/emiliano.penaloza/LLM4REC/results/books/gpt4_results_19_up.csv')

In [ ]:
df.median()